# Basics: Data Loading & GPS Visualization

This notebook demonstrates the fundamentals of loading and visualizing AIM™ telemetry data using libxrk.

## What You'll Find Here

- **Data Loading**: Load `.xrk` or `.xrz` files without AIM software
- **Lap Times Table**: View all recorded laps with computed lap times
- **GPS Speed Map**: Visualize speed around the track on an interactive map
- **Brake & Throttle Overlay**: See driver inputs overlaid on the track map

## Using Your Own Data

To analyze your own data:

1. **Run the first cell** below to install packages and display the upload widget
2. **Click "Choose File"** to select your `.xrk` or `.xrz` file
3. **Run all remaining cells** to analyze your data

The status indicator will show which file is being used. If you don't upload a file, the sample data will be used.

## Requirements

- GPS data channels (`GPS Latitude`, `GPS Longitude`, `GPS Speed`)
- Brake pressure (`BrakePress`) and throttle (`PPS`) for the inputs overlay

**Note:** This notebook works in both JupyterLite (browser) and standard JupyterLab environments.

In [1]:
# Install required packages (needed for JupyterLite, skipped in regular JupyterLab if already installed)
%pip install -q pandas plotly libxrk motorsports-data-notebook jinja2 ipywidgets

# Import helper functions
from motorsports_data_notebook.visualization import (
    format_lap_time,
    plot_gps_channels,
    show_fig,
)
from motorsports_data_notebook.widgets import SessionPicker

# Session picker - upload your own file and select a lap to analyze
session = SessionPicker(default_file="CMD_Inferno 86_Fuji GP Sh_Generic testing_a_2248.xrz")
session.display()

Note: you may need to restart the kernel to use updated packages.


/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:343: RuntimeWarning: invalid value encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:356: RuntimeWarning: divide by zero encountered in divide
  t = np.maximum(-np.sum(SN * O, axis=1) / np.sum(SN * D, axis=1), 0)
/home/runner/.cache/pypoetry/virtualenvs/motorsports-data-notebook-lSQWGACw-py3.12/lib/python3.12/site-packages/libxrk/gps.py:357: RuntimeWarning: invalid value encountered in multiply
  dist = np.sum(np.square(O + t.reshape((len(t), 1)) * D), axis=1)


division: scan=0.102432, gps=0.043779, group/ch=0.069211 more


In [2]:
# Get laps as pandas DataFrame for display
laps = session.get_laps()

In [3]:
# Display lap times table
laps.style.format({"lap_time": format_lap_time})  # type: ignore[dict-item]

,num,start_time,end_time,lap_time
0,0,0,150454,2:30.454
1,1,150454,279602,2:09.148
2,2,279602,406240,2:06.638
3,3,406240,532797,2:06.557
4,4,532797,659282,2:06.485
5,5,659282,787773,2:08.491
6,6,787773,913776,2:06.003
7,7,913776,1041397,2:07.621
8,8,1041397,1168322,2:06.925
9,9,1168322,1294676,2:06.354


In [4]:
# Extract channel data for the selected lap using libxrk 0.5.0 methods
selected_lap = session.get_selected_lap()
log = session.get_log()
lap_num = int(selected_lap["num"])

# Filter to lap, select channels, and resample to GPS timebase
channels = (
    log.filter_by_lap(lap_num)
    .select_channels(["GPS Latitude", "GPS Longitude", "speed_kmh", "BrakePress", "PPS"])
    .resample_to_channel("GPS Latitude")
    .channels
)

In [5]:
# Plot speed on GPS map using channel tables directly
fig = plot_gps_channels(
    channels,
    lat_channel="GPS Latitude",
    lon_channel="GPS Longitude",
    color_channels=[("speed_kmh", "Speed (km/h)", "Viridis")],
    title=f"Speed - Lap {int(selected_lap['num'])}",
)
show_fig(fig)

In [6]:
# Plot with multiple color channels (automatically interpolated to GPS timebase)
fig = plot_gps_channels(
    channels,
    lat_channel="GPS Latitude",
    lon_channel="GPS Longitude",
    color_channels=[
        ("BrakePress", "BrakePres", "Reds"),
        ("PPS", "Throttle", "Greens"),
    ],
    title=f"Accelerator and Brake Pressure - Lap {int(selected_lap['num'])}",
)
show_fig(fig)